In [ ]:
import os
import copy
import hist
from coffea import util
import numpy as np
import itertools
import pandas as pd
import mplhep as hep

In [ ]:
btagDir = 'MediumBTag/'#'LooseBTag/'
yearDir = '2016/'
APVDir = 'noAPV/'
Era = ''
od = ''
vfp = 'postVFP'
if APVDir == 'APV/':
    vfp = 'preVFP'
if btagDir == '':
    od = '_oldANdisc'

In [ ]:
# JetHT2016_unweighted = util.load('CoffeaOutputsForCombine/Coffea_FirstRun/JetHT/' + btagDir + yearDir + 'TTbarRes_0l_JetHT2016_Data' + od + '.coffea')
JetHT2016_unweighted = util.load('CoffeaOutputsForCombine/Coffea_FirstRun/JetHT/' + btagDir + yearDir + APVDir +'TTbarRes_0l_UL16'+vfp+'_JetHT' + Era + '_Data' + od + '.coffea')
TTbar_unweighted = util.load('CoffeaOutputsForCombine/Coffea_FirstRun/TT/' + btagDir + yearDir + APVDir +'TTbarRes_0l_UL16'+vfp+'_TTbar' + od + '.coffea')

In [ ]:
Datasets = {
    'JetHT 2016': JetHT2016_unweighted,
#     'JetHT 2017': JetHT2017_unweighted,
#     'JetHT 2018': JetHT2018_unweighted,
#     'JetHT All': JetHT_unweighted,
    'TTbar': TTbar_unweighted
    }

# Cutflow

In [ ]:
for name,output in Datasets.items(): 
    print("-------Unweighted " + name + "--------")
    for i,j in output['cutflow'].items():        
        print( '%20s : %12d' % (i,j) )

In [ ]:
def mkdir_p(mypath):
    '''Creates a directory. equivalent to using mkdir -p on the command line'''

    from errno import EEXIST
    from os import makedirs,path

    try:
        makedirs(mypath)
    except OSError as exc: # Python >2.5
        if exc.errno == EEXIST and path.isdir(mypath):
            pass
        else: raise

In [ ]:
def DoesDirectoryExist(mypath): #extra precaution (Probably overkill...)
    '''Checks to see if Directory exists before running mkdir_p'''
    import os.path
    from os import path
    
    if path.exists(mypath):
        pass
    else:
        mkdir_p(mypath)

In [ ]:
def plotratio(numerator, denominator, ax=None, histtype='errorbar', marker='.', markersize=5., color='k', alpha=0.1):
    NumeratorAxes = numerator.axes
    DenominatorAxes = denominator.axes
    
    # integer number of bins in this axis #
    NumeratorAxis1_BinNumber = NumeratorAxes[0].size - 3 # Subtract 3 to remove overflow
    
    DenominatorAxis1_BinNumber = DenominatorAxes[0].size - 3 
    
    if(NumeratorAxis1_BinNumber != DenominatorAxis1_BinNumber):
        raise Exception('Numerator and Denominator axes are different sizes; Cannot perform division.')
    # else:
    #     Numerator = numerator.to_hist()
    #     Denominator = denominator.to_hist()
        
    ratio = numerator / denominator.values()
    
    if histtype == 'errorbar':
        return hep.histplot(ratio, ax=ax, histtype=histtype, marker=marker, markersize=markersize, color=color)
    elif histtype == 'fill':
        return hep.histplot(ratio, ax=ax, histtype=histtype, color=color, alpha=alpha, lw=5.)
    else:
        return hep.histplot(ratio, ax=ax, histtype=histtype, color=color)

In [ ]:
def ConvertLabelToInt(mapping, str_label):
    for intkey, string in mapping.items():
        if str_label == string:
            return intkey

In [ ]:
import matplotlib.pyplot as plt
import warnings
import re # regular expressions
warnings.filterwarnings("ignore")

# ---- Reiterate categories ---- #
ttagcats = ["at"] #, "0t", "1t", "It", "2t"]
btagcats = ["0b", "1b", "2b"]
ycats = ['cen', 'fwd']

list_of_cats = [ t+b+y for t,b,y in itertools.product( ttagcats, btagcats, ycats) ]
list_of_ints = [6, 7, 8, 9, 10, 11]
catmap = dict(zip(list_of_ints, list_of_cats))

In [ ]:
maindirectory = os.getcwd() 
print(maindirectory)

In [ ]:
stack_ttbar_opts = {'alpha': 0.8, 'edgecolor':(0,0,0,0.3), 'color': 'red'}
stack_background_opts = {'alpha': 0.8, 'edgecolor':(0,0,0,0.3), 'color': 'yellow'}
stack_error_opts = {'label':'Stat. Unc.', 'hatch':'///', 'facecolor':'None', 'edgecolor':(0,0,0,.5), 'linewidth': 0}
data_err_opts = {'linestyle': 'none', 'marker': '.', 'markersize': 10., 'color': 'k', 'elinewidth': 1}

# Luminosities, Cross Sections & Scale Factors for MC

In [ ]:
Nevts2016 = 625441538 # from dasgoclient
Nevts2016_sf = Nevts2016/JetHT2016_unweighted['cutflow']['all events']

# Nevts2017 = 410461585.
# Nevts2017_sf = Nevts2017/JetHT2017_unweighted['cutflow']['all events']

# Nevts2018 = 676328827.
# Nevts2018_sf = Nevts2018/JetHT2018_unweighted['cutflow']['all events']

# NevtsAll = Nevts2016+Nevts2017+Nevts2018
# Nevts_sf = NevtsAll/JetHT_unweighted['cutflow']['all events']

print(Nevts2016_sf)

In [ ]:
Lum2016 = 35920./Nevts2016_sf # pb^-1 from https://twiki.cern.ch/twiki/bin/viewauth/CMS/PdmVAnalysisSummaryTable
Lum2017 = 41530./3.
Lum2018 = 59740./3.
Lum     = 137190./3.

ttbar_BR = 0.4544 #PDG 2019
ttbar_xs = 831.76  #pb  Monte Carlo already includes xs in event weight!!
toptag_kf = 0.49 # nominal
toptag_kf_up = 0.70
toptag_kf_down = 0.28

ttbar2016_sf = ttbar_BR*toptag_kf*Lum2016/TTbar_unweighted['cutflow']['all events']  #ttbar_xs*Lum2016/142155064.
ttbar2017_sf = ttbar_xs*Lum2017/TTbar_unweighted['cutflow']['sumw']  #ttbar_xs*Lum2017/142155064.
ttbar2018_sf = ttbar_xs*Lum2018/TTbar_unweighted['cutflow']['sumw']  #ttbar_xs*Lum2018/142155064.
ttbar_sf = ttbar_xs*Lum/142155064.


print(r'$t\bar{t}$ 2016 scale factor = ', ttbar2016_sf)

In [ ]:
# ---- Optional to rescale x-axis of mistag rates ---- #
def forward(x):
    return x**(1/8)

def inverse(x):
    return x**8

# Mistag Rates with $t\bar{t}$ Subtraction for SeparateYears

In [ ]:
SaveDirectory = maindirectory + '/' + 'MistagPlots' + '/' # split histograms into subdirectories
DoesDirectoryExist(SaveDirectory) # no need to create the directory several times if it exists already
for icat in list_of_ints:
        
    title = 'mistag ' + catmap[icat][2:]
    
    # ---- define ttbar numerator and denominator ---- #
    NumeratorTT2016 = TTbar_unweighted['numerator']['UL16'+vfp+'_TTbar', icat, :]
    DenominatorTT2016 = TTbar_unweighted['denominator']['UL16'+vfp+'_TTbar', icat, :]
    
    # ---- scale ttbar and prepare to subtract ---- #
    NumeratorTT2016 *= (-ttbar2016_sf)
    DenominatorTT2016 *= (-ttbar2016_sf)
    
    # ---- define data numerator and denominator ---- #
    Numerator2016 = JetHT2016_unweighted['numerator']['UL16'+vfp+'_JetHT_Data', icat, :]
    Denominator2016 = JetHT2016_unweighted['denominator']['UL16'+vfp+'_JetHT_Data', icat, :]

#     N_vals = Numerator2016.view().value
#     D_vals = Denominator2016.view().value
#     print(f'{icat}, {catmap[icat]}\n')
#     print(f'Numerator/Denominator = \n\n{N_vals}\n-----------------------------------\n{D_vals}\n\n===================\n\n')
    
    # ---- scale data ---- #
    Numerator2016 *= (Nevts2016_sf)
    Denominator2016 *= (Nevts2016_sf)
    
    # ---- add (remove) ttbar mc ---- #
    Numerator2016 += (NumeratorTT2016)
    Denominator2016 += (DenominatorTT2016)

    # ==== Scikit plot_ratio function ==== #
#     fig = plt.figure(figsize=(10, 8))
#     main_ax_artists, sublot_ax_arists = Numerator2016.plot_ratio(
#         Denominator2016,
#         rp_ylabel=r"Ratio",
#         rp_num_label="num",
#         rp_denom_label="denom",
#         rp_uncert_draw_type="bar")
    
    # ==== My function (Closer to coffea.hist.plotratio) ==== #
    fig, ax = plt.subplots(
        #nrows=2,
        #ncols=1,
        figsize=(7,5),
        #gridspec_kw={"height_ratios": (3, 1)},
        sharex=True
    )
    mistag2016 = plotratio(Numerator2016, Denominator2016, ax = ax, 
                           histtype='errorbar', marker='.', markersize=13., color='r')


    ax.set_ylabel('Mistag Rates')
    ax.set_title(title)
    leg = ax.legend(labels=["2016", "2017", "2018"])

    # ---- Optional x-axis scaling ---- #
    plt.ylim(bottom = 0, top = 0.10)
    plt.xlim(left = 100, right = 3000)

    # ---- Optional x-axis scaling ---- #
    #ax.set_xscale('function', functions=(forward, inverse))
    #plt.xticks(np.array([0, 500, 1000, 2000, 3000, 4000, 5000]))

    # ---- Optional x-axis scaling ---- #
    #ax.set_xscale('function', functions=(forward, inverse))
    #plt.xticks(np.array([500, 1000, 2000, 5000, 10000]))
    #plt.yticks(np.array([.05, .10, .15]))
    #ax.set_xscale('log')

    #filename = 'Mistag_bdisc8484_ttbarSubtraction_SeparateYears_' + icat + '.png'
    #plt.savefig(SaveDirectory+filename, bbox_inches="tight")
    #print('\n' + filename + ' saved')

# Mistag Rates with $t\bar{t}$ Subtraction for SeparateYears with Inclusive $y$ Region

In [ ]:
""" ---------- Rapidity Inclusive Mistag Rates ---------- """
SaveDirectory = maindirectory + '/' + 'MistagPlots' + '/' # split histograms into subdirectories
DoesDirectoryExist(SaveDirectory) # no need to create the directory several times if it exists already

for b in range(3):

    title = f'Inclusive {b}btag Mistag Rate'
    
    NumeratorTT2016cen = TTbar_unweighted['numerator']['UL16'+vfp+'_TTbar', 6+2*b, :]
    DenominatorTT2016cen = TTbar_unweighted['denominator']['UL16'+vfp+'_TTbar', 6+2*b, :]
    NumeratorTT2016fwd = TTbar_unweighted['numerator']['UL16'+vfp+'_TTbar', 7+2*b, :]
    DenominatorTT2016fwd = TTbar_unweighted['denominator']['UL16'+vfp+'_TTbar', 7+2*b, :]

    
    NumeratorTT2016cen *= (-ttbar2016_sf)
    DenominatorTT2016cen *= (-ttbar2016_sf)
    NumeratorTT2016fwd *= (-ttbar2016_sf)
    DenominatorTT2016fwd *= (-ttbar2016_sf)
    
    
    Num_cen2016 = JetHT2016_unweighted['numerator'][f'UL16{vfp}_JetHT{Era}_Data', 6+2*b, :]
    Denom_cen2016 = JetHT2016_unweighted['denominator'][f'UL16{vfp}_JetHT{Era}_Data', 6+2*b, :]
    Num_cen2016 *= (Nevts2016_sf)
    Denom_cen2016 *= (Nevts2016_sf)
    Num_cen2016 += (NumeratorTT2016cen)
    Denom_cen2016 += (DenominatorTT2016cen)

    Num_fwd2016 = JetHT2016_unweighted['numerator'][f'UL16{vfp}_JetHT{Era}_Data', 7+2*b, :]
    Denom_fwd2016 = JetHT2016_unweighted['denominator'][f'UL16{vfp}_JetHT{Era}_Data', 7+2*b, :]
    Num_fwd2016 *= (Nevts2016_sf)
    Denom_fwd2016 *= (Nevts2016_sf)
    Num_fwd2016 += (NumeratorTT2016fwd)
    Denom_fwd2016 += (DenominatorTT2016fwd)
    
    Num_inc2016 = Num_cen2016
    Denom_inc2016 = Denom_cen2016
    
    Num_inc2016 += (Num_fwd2016)
    Denom_inc2016 += (Denom_fwd2016)
        
#     fig = plt.figure(figsize=(10, 8))
#     main_ax_artists, sublot_ax_arists = Num_inc2016.plot_ratio(
#         Denom_inc2016,
#         rp_ylabel=r"Ratio",
#         rp_num_label="num",
#         rp_denom_label="denom",
#         rp_uncert_draw_type="bar")

    fig, ax = plt.subplots(
        figsize=(10,5),
        sharex=True
    )

    mistag2016 = plotratio(Num_inc2016, Denom_inc2016, ax = ax, histtype='errorbar', 
                           marker='.', markersize=13., color='r')


    ax.set_ylabel('Mistag Rates')
    ax.set_title(title)
    leg = ax.legend(labels=["2016", "2017", "2018"])

    plt.ylim(bottom = 0, top = 0.08)
    plt.xlim(left = 400, right = 7000)

    # ---- Optional x-axis scaling ---- #
    ax.set_xscale('function', functions=(forward, inverse))
    plt.xticks(np.array([400, 500, 1000, 2000, 3000, 4000, 5000]))
    ax.set_xticklabels(['', 500, 1000, 2000, 3000, 4000, 5000])
    # ---- Optional x-axis scaling ---- #
    #ax.set_xscale('function', functions=(forward, inverse))
    #plt.xticks(np.array([500, 1000, 2000, 5000, 10000]))
    #plt.yticks(np.array([.05, .10, .15]))
    #ax.set_xscale('log')
    
    #filename = 'InclusiveMistag_bdisc8484_ttbarSubtraction_SeparateYears_' + btag + '.png'
    #plt.savefig(SaveDirectory+filename, bbox_inches="tight")
    #print('\n' + filename + ' saved')

# Mistag Rates with and without $t\bar{t}$ Subtraction for 2016 Data with Inclusive $y$ Region Sorted by b-tag Region

In [ ]:
### COPY THIS CELL FOR OTHER YEARS ###
SaveDirectory = maindirectory + '/' + 'MistagPlots/' + btagDir + yearDir + APVDir # split histograms into subdirectories
DoesDirectoryExist(SaveDirectory) # no need to create the directory several times if it exists already

fig, ax = plt.subplots(
    figsize=(10,5),
    sharex=True
)
HIP = 'No HIP'
if vfp == 'preVFP':
    HIP = 'HIP'
title = f'2016 ({HIP}) Mistag Rate'

NumeratorTTcen0b = TTbar_unweighted['numerator'][f'UL16{vfp}_TTbar', 6, :]
DenominatorTTcen0b = TTbar_unweighted['denominator'][f'UL16{vfp}_TTbar', 6, :]
NumeratorTTfwd0b = TTbar_unweighted['numerator'][f'UL16{vfp}_TTbar', 7, :]
DenominatorTTfwd0b = TTbar_unweighted['denominator'][f'UL16{vfp}_TTbar', 7, :]
NumeratorTTcen1b = TTbar_unweighted['numerator'][f'UL16{vfp}_TTbar', 8, :]
DenominatorTTcen1b = TTbar_unweighted['denominator'][f'UL16{vfp}_TTbar', 8, :]
NumeratorTTfwd1b = TTbar_unweighted['numerator'][f'UL16{vfp}_TTbar', 9, :]
DenominatorTTfwd1b = TTbar_unweighted['denominator'][f'UL16{vfp}_TTbar', 9, :]
NumeratorTTcen2b = TTbar_unweighted['numerator'][f'UL16{vfp}_TTbar', 10, :]
DenominatorTTcen2b = TTbar_unweighted['denominator'][f'UL16{vfp}_TTbar', 10, :]
NumeratorTTfwd2b = TTbar_unweighted['numerator'][f'UL16{vfp}_TTbar', 11, :]
DenominatorTTfwd2b = TTbar_unweighted['denominator'][f'UL16{vfp}_TTbar', 11, :]

NumeratorTTcen0b *= (-ttbar2016_sf)
DenominatorTTcen0b *= (-ttbar2016_sf)
NumeratorTTfwd0b *= (-ttbar2016_sf)
DenominatorTTfwd0b *= (-ttbar2016_sf)
NumeratorTTcen1b *= (-ttbar2016_sf)
DenominatorTTcen1b *= (-ttbar2016_sf)
NumeratorTTfwd1b *= (-ttbar2016_sf)
DenominatorTTfwd1b *= (-ttbar2016_sf)
NumeratorTTcen2b *= (-ttbar2016_sf)
DenominatorTTcen2b *= (-ttbar2016_sf)
NumeratorTTfwd2b *= (-ttbar2016_sf)
DenominatorTTfwd2b *= (-ttbar2016_sf)
    


Num_cen0b = JetHT2016_unweighted['numerator'][f'UL16{vfp}_JetHT{Era}_Data', 6, :]
Denom_cen0b = JetHT2016_unweighted['denominator'][f'UL16{vfp}_JetHT{Era}_Data', 6, :]
Num_cen0b *= (Nevts2016_sf)
Denom_cen0b *= (Nevts2016_sf)
# Num_cen0b += (NumeratorTTcen0b)
# Denom_cen0b += (DenominatorTTcen0b)

Num_fwd0b = JetHT2016_unweighted['numerator'][f'UL16{vfp}_JetHT{Era}_Data', 7, :]
Denom_fwd0b = JetHT2016_unweighted['denominator'][f'UL16{vfp}_JetHT{Era}_Data', 7, :]
Num_fwd0b *= (Nevts2016_sf)
Denom_fwd0b *= (Nevts2016_sf)
# Num_fwd0b += (NumeratorTTfwd0b)
# Denom_fwd0b += (DenominatorTTfwd0b)

Num_inc0b = Num_cen0b
Denom_inc0b = Denom_cen0b

Num_inc0b += (Num_fwd0b)
Denom_inc0b += (Denom_fwd0b)

mistag_inclusive0b = plotratio(Num_inc0b, Denom_inc0b, ax = ax, histtype='errorbar', 
                           marker='.', markersize=13., color='r')

Num_cen1b = JetHT2016_unweighted['numerator'][f'UL16{vfp}_JetHT{Era}_Data', 8, :]
Denom_cen1b = JetHT2016_unweighted['denominator'][f'UL16{vfp}_JetHT{Era}_Data', 8, :]
Num_cen1b *= (Nevts2016_sf)
Denom_cen1b *= (Nevts2016_sf)
# Num_cen1b += (NumeratorTTcen1b)
# Denom_cen1b +=(DenominatorTTcen1b)

Num_fwd1b = JetHT2016_unweighted['numerator'][f'UL16{vfp}_JetHT{Era}_Data', 9, :]
Denom_fwd1b = JetHT2016_unweighted['denominator'][f'UL16{vfp}_JetHT{Era}_Data', 9, :]
Num_fwd1b *= (Nevts2016_sf)
Denom_fwd1b *= (Nevts2016_sf)
# Num_fwd1b += (NumeratorTTfwd1b)
# Denom_fwd1b += (DenominatorTTfwd1b)

Num_inc1b = Num_cen1b
Denom_inc1b = Denom_cen1b

Num_inc1b += (Num_fwd1b)
Denom_inc1b += (Denom_fwd1b)

mistag_inclusive1b = plotratio(Num_inc1b, Denom_inc1b, ax = ax, histtype='errorbar', 
                           marker='s', markersize=7., color='g')

Num_cen2b = JetHT2016_unweighted['numerator'][f'UL16{vfp}_JetHT{Era}_Data', 10, :]
Denom_cen2b = JetHT2016_unweighted['denominator'][f'UL16{vfp}_JetHT{Era}_Data', 10, :]
Num_cen2b *= (Nevts2016_sf)
Denom_cen2b *= (Nevts2016_sf)
# Num_cen2b += (NumeratorTTcen2b)
# Denom_cen2b += (DenominatorTTcen2b)

Num_fwd2b = JetHT2016_unweighted['numerator'][f'UL16{vfp}_JetHT{Era}_Data', 11, :]
Denom_fwd2b = JetHT2016_unweighted['denominator'][f'UL16{vfp}_JetHT{Era}_Data', 11, :]
Num_fwd2b *= (Nevts2016_sf)
Denom_fwd2b *= (Nevts2016_sf)
# Num_fwd2b += (NumeratorTTfwd2b)
# Denom_fwd2b += (DenominatorTTfwd2b)

Num_inc2b = Num_cen2b
Denom_inc2b = Denom_cen2b

Num_inc2b += (Num_fwd2b)
Denom_inc2b += (Denom_fwd2b)

mistag_inclusive2b = plotratio(Num_inc2b, Denom_inc2b, ax = ax, histtype='errorbar', 
                           marker='^', markersize=7., color='b')

plt.ylim(bottom = 0, top = 0.10)
plt.xlim([400,7000])

ax.set_ylabel('t-tag Mistag Rates')
ax.set_title(title)
leg = ax.legend(labels=["0b", "1b", "2b"],fontsize='x-large')

# ---- Optional x-axis scaling ---- #
ax.set_xscale('function', functions=(forward, inverse))
plt.xticks(np.array([400, 500, 600, 700, 800, 900, 1000, 2000, 3000, 4000, 5000]))
ax.set_xticklabels(['', 500, '', '', '', '', 1000, 2000, 3000, 4000, 5000])
# plt.yticks(np.array([.05, .10]))
#ax.set_xscale('log')

#---- Labeling ----#
Lint = str(Lum2016*.001) # Integrated Luminosity
lumi = plt.text(1.0, 1.08, "L = " + Lint[:6] + " fb$^{-1}$",
        fontsize='xx-large',
        horizontalalignment='right',
        verticalalignment='top',
        transform=ax.transAxes
       )
CMS = plt.text(0.01, 0.98, 'CMS Preliminary',
        fontsize='xx-large',
        horizontalalignment='left',
        verticalalignment='top',
        transform=ax.transAxes
       )
rap = plt.text(0.05, 0.86, 'y-inclusive',
        fontsize='large',
        horizontalalignment='left',
        verticalalignment='top',
        transform=ax.transAxes
       )
coffee = plt.text(1.0, 0.87, u"☕",
              fontsize=50,
              horizontalalignment='left',
              verticalalignment='bottom',
              transform=ax.transAxes
             )

# filename = 'InclusiveMistag.png'
# plt.savefig(SaveDirectory+filename, bbox_inches="tight")
# print('\n' + SaveDirectory + filename + ' saved')

In [ ]:
### COPY THIS CELL FOR OTHER YEARS ###
SaveDirectory = maindirectory + '/' + 'MistagPlots/' + btagDir + yearDir + APVDir # split histograms into subdirectories
DoesDirectoryExist(SaveDirectory) # no need to create the directory several times if it exists already

fig, ax = plt.subplots(
    figsize=(10,5),
    sharex=True
)
HIP = 'No HIP'
if vfp == 'preVFP':
    HIP = 'HIP'
title = f'2016 ({HIP}) Mistag Rate'

NumeratorTTcen0b = TTbar_unweighted['numerator'][f'UL16{vfp}_TTbar', 6, :]
DenominatorTTcen0b = TTbar_unweighted['denominator'][f'UL16{vfp}_TTbar', 6, :]
NumeratorTTfwd0b = TTbar_unweighted['numerator'][f'UL16{vfp}_TTbar', 7, :]
DenominatorTTfwd0b = TTbar_unweighted['denominator'][f'UL16{vfp}_TTbar', 7, :]
NumeratorTTcen1b = TTbar_unweighted['numerator'][f'UL16{vfp}_TTbar', 8, :]
DenominatorTTcen1b = TTbar_unweighted['denominator'][f'UL16{vfp}_TTbar', 8, :]
NumeratorTTfwd1b = TTbar_unweighted['numerator'][f'UL16{vfp}_TTbar', 9, :]
DenominatorTTfwd1b = TTbar_unweighted['denominator'][f'UL16{vfp}_TTbar', 9, :]
NumeratorTTcen2b = TTbar_unweighted['numerator'][f'UL16{vfp}_TTbar', 10, :]
DenominatorTTcen2b = TTbar_unweighted['denominator'][f'UL16{vfp}_TTbar', 10, :]
NumeratorTTfwd2b = TTbar_unweighted['numerator'][f'UL16{vfp}_TTbar', 11, :]
DenominatorTTfwd2b = TTbar_unweighted['denominator'][f'UL16{vfp}_TTbar', 11, :]

NumeratorTTcen0b *= (-ttbar2016_sf)
DenominatorTTcen0b *= (-ttbar2016_sf)
NumeratorTTfwd0b *= (-ttbar2016_sf)
DenominatorTTfwd0b *= (-ttbar2016_sf)
NumeratorTTcen1b *= (-ttbar2016_sf)
DenominatorTTcen1b *= (-ttbar2016_sf)
NumeratorTTfwd1b *= (-ttbar2016_sf)
DenominatorTTfwd1b *= (-ttbar2016_sf)
NumeratorTTcen2b *= (-ttbar2016_sf)
DenominatorTTcen2b *= (-ttbar2016_sf)
NumeratorTTfwd2b *= (-ttbar2016_sf)
DenominatorTTfwd2b *= (-ttbar2016_sf)
    


Num_cen0b = JetHT2016_unweighted['numerator'][f'UL16{vfp}_JetHT{Era}_Data', 6, :]
Denom_cen0b = JetHT2016_unweighted['denominator'][f'UL16{vfp}_JetHT{Era}_Data', 6, :]
Num_cen0b *= (Nevts2016_sf)
Denom_cen0b *= (Nevts2016_sf)
Num_cen0b += (NumeratorTTcen0b)
Denom_cen0b += (DenominatorTTcen0b)

Num_fwd0b = JetHT2016_unweighted['numerator'][f'UL16{vfp}_JetHT{Era}_Data', 7, :]
Denom_fwd0b = JetHT2016_unweighted['denominator'][f'UL16{vfp}_JetHT{Era}_Data', 7, :]
Num_fwd0b *= (Nevts2016_sf)
Denom_fwd0b *= (Nevts2016_sf)
Num_fwd0b += (NumeratorTTfwd0b)
Denom_fwd0b += (DenominatorTTfwd0b)

Num_inc0b = Num_cen0b
Denom_inc0b = Denom_cen0b

Num_inc0b += (Num_fwd0b)
Denom_inc0b += (Denom_fwd0b)

mistag_inclusive0b = plotratio(Num_inc0b, Denom_inc0b, ax = ax, histtype='errorbar', 
                           marker='.', markersize=13., color='r')

Num_cen1b = JetHT2016_unweighted['numerator'][f'UL16{vfp}_JetHT{Era}_Data', 8, :]
Denom_cen1b = JetHT2016_unweighted['denominator'][f'UL16{vfp}_JetHT{Era}_Data', 8, :]
Num_cen1b *= (Nevts2016_sf)
Denom_cen1b *= (Nevts2016_sf)
Num_cen1b += (NumeratorTTcen1b)
Denom_cen1b +=(DenominatorTTcen1b)

Num_fwd1b = JetHT2016_unweighted['numerator'][f'UL16{vfp}_JetHT{Era}_Data', 9, :]
Denom_fwd1b = JetHT2016_unweighted['denominator'][f'UL16{vfp}_JetHT{Era}_Data', 9, :]
Num_fwd1b *= (Nevts2016_sf)
Denom_fwd1b *= (Nevts2016_sf)
Num_fwd1b += (NumeratorTTfwd1b)
Denom_fwd1b += (DenominatorTTfwd1b)

Num_inc1b = Num_cen1b
Denom_inc1b = Denom_cen1b

Num_inc1b += (Num_fwd1b)
Denom_inc1b += (Denom_fwd1b)

mistag_inclusive1b = plotratio(Num_inc1b, Denom_inc1b, ax = ax, histtype='errorbar', 
                           marker='s', markersize=7., color='g')

Num_cen2b = JetHT2016_unweighted['numerator'][f'UL16{vfp}_JetHT{Era}_Data', 10, :]
Denom_cen2b = JetHT2016_unweighted['denominator'][f'UL16{vfp}_JetHT{Era}_Data', 10, :]
Num_cen2b *= (Nevts2016_sf)
Denom_cen2b *= (Nevts2016_sf)
Num_cen2b += (NumeratorTTcen2b)
Denom_cen2b += (DenominatorTTcen2b)

Num_fwd2b = JetHT2016_unweighted['numerator'][f'UL16{vfp}_JetHT{Era}_Data', 11, :]
Denom_fwd2b = JetHT2016_unweighted['denominator'][f'UL16{vfp}_JetHT{Era}_Data', 11, :]
Num_fwd2b *= (Nevts2016_sf)
Denom_fwd2b *= (Nevts2016_sf)
Num_fwd2b += (NumeratorTTfwd2b)
Denom_fwd2b += (DenominatorTTfwd2b)

Num_inc2b = Num_cen2b
Denom_inc2b = Denom_cen2b

Num_inc2b += (Num_fwd2b)
Denom_inc2b += (Denom_fwd2b)

mistag_inclusive2b = plotratio(Num_inc2b, Denom_inc2b, ax = ax, histtype='errorbar', 
                           marker='^', markersize=7., color='b')

plt.ylim(bottom = 0, top = 0.10)
plt.xlim([400,7000])

ax.set_ylabel('t-tag Mistag Rate')
ax.set_title(title)
leg = ax.legend(labels=["0b", "1b", "2b"],fontsize='x-large')

# ---- Optional x-axis scaling ---- #
ax.set_xscale('function', functions=(forward, inverse))
plt.xticks(np.array([400, 500, 600, 700, 800, 900, 1000, 2000, 3000, 4000, 5000]))
ax.set_xticklabels(['', 500, '', '', '', '', 1000, 2000, 3000, 4000, 5000])
# plt.yticks(np.array([.05, .10]))
#ax.set_xscale('log')

#---- Labeling ----#
Lint = str(Lum2016*.001) # Integrated Luminosity
lumi = plt.text(1.0, 1.08, "L = " + Lint[:6] + " fb$^{-1}$",
        fontsize='xx-large',
        horizontalalignment='right',
        verticalalignment='top',
        transform=ax.transAxes
       )
CMS = plt.text(0.01, 0.98, 'CMS Preliminary',
        fontsize='xx-large',
        horizontalalignment='left',
        verticalalignment='top',
        transform=ax.transAxes
       )
rap = plt.text(0.05, 0.86, 'y-inclusive',
        fontsize='large',
        horizontalalignment='left',
        verticalalignment='top',
        transform=ax.transAxes
       )
contam = plt.text(0.05, 0.81, r'$t\bar{t}$ contam. removed',
        fontsize='large',
        horizontalalignment='left',
        verticalalignment='top',
        transform=ax.transAxes
       )
coffee = plt.text(1.0, 0.87, u"☕",
              fontsize=50,
              horizontalalignment='left',
              verticalalignment='bottom',
              transform=ax.transAxes
             )

# filename = 'InclusiveMistag.png'
# plt.savefig(SaveDirectory+filename, bbox_inches="tight")
# print('\n' + SaveDirectory + filename + ' saved')

# Mistag Rates with $t\bar{t}$ Subtraction for All Years Combined with Inclusive $y$ Region